In [20]:
from ds_utils.imports import *

Using Theano backend.


# Prework

Following this fast.ai [lesson](https://www.youtube.com/watch?v=V2h3IOBDvrA&feature=youtu.be&t=5761).

In [ ]:
# get data
# !wget -O data/ml-latest-small.zip http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !unzip data/ml-latest-small.zip

# Preprocessing

In [38]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

In [39]:
ratings.sample(5)

,userId,movieId,rating,timestamp
25228,185,1028,3.0,1003524590
55514,402,1196,4.5,1443393714
15952,102,3424,5.0,956591391
35465,254,234,3.0,845157610
66037,468,1953,3.0,1296189259


In [45]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,346.011310,1660.778349,3.543608,1.129639e+09
std,195.163838,1893.955817,1.058064,1.916858e+08
min,0.000000,0.000000,0.500000,7.896520e+08
25%,181.000000,327.000000,3.000000,9.658478e+08
50%,366.000000,873.000000,4.000000,1.110422e+09
75%,519.000000,2344.000000,4.000000,1.296192e+09
max,670.000000,9065.000000,5.000000,1.476641e+09


In [41]:
ratings.shape

(100004, 4)

In [42]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [43]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [44]:
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])

In [46]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [49]:
n_factors = 50

In [50]:
np.random.seed = 42

In [55]:
msk = np.random.rand(len(ratings))<0.8
trn = ratings[msk]
val = ratings[~msk]

# excel like table

In [60]:
g = ratings.groupby('userId')['rating'].count()

In [62]:
topUsers = g.sort_values(ascending=False)[:15]

In [65]:
g = ratings.groupby('movieId')['rating'].count()

In [66]:
topMovies = g.sort_values(ascending=False)[:15]

In [72]:
top_r = ratings.join(topUsers, how='inner', on='userId', rsuffix='_r')

In [74]:
top_r = top_r.join(topMovies, how='inner', on='movieId', rsuffix='_r')

In [76]:
top_r.head()

,userId,movieId,rating,timestamp,rating_r,rating_r
962,14,417,2.0,997938310,1700,247
5048,29,417,4.0,944943070,1011,247
10214,72,417,5.0,1303464840,1610,247
28390,211,417,3.0,1218405007,876,247
29266,212,417,3.0,1462637445,910,247


In [79]:
pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum)

movieId,0,1,2,3,4,5,6,7,8,9,...,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
userId,,,,,,,,,,,,,,,,,,,,,
0,2.5,3.0,3.0,2.0,4.0,2.0,2.0,2.0,3.5,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


# dot product